In [27]:
import pandas as pd
import json
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import nltk
from nltk.tokenize import word_tokenize
import spacy
from itertools import permutations
nlp = spacy.load('en_core_web_sm')

In [2]:
original = pd.read_csv('~/Downloads/wikihowSep.csv')
original.head()

,overview,headline,text,sectionLabel,title
0,So you're a new or aspiring artist and your c...,\nSell yourself first.,"Before doing anything else, stop and sum up y...",Steps,How to Sell Fine Art Online
1,"If you want to be well-read, then, in the wor...",\nRead the classics before 1600.,Reading the classics is the very first thing ...,Reading the Classics,How to Be Well Read
2,So you're a new or aspiring artist and your c...,\nJoin online artist communities.,Depending on what scale you intend to sell yo...,Steps,How to Sell Fine Art Online
3,So you're a new or aspiring artist and your c...,\nMake yourself public.,Get yourself out there as best as you can by ...,Steps,How to Sell Fine Art Online
4,So you're a new or aspiring artist and your c...,\nBlog about your artwork.,"Given the hundreds of free blogging websites,...",Steps,How to Sell Fine Art Online


In [4]:
# drop rows where at least 1 element is missing
df = original.dropna()
df = df[:1000]

In [6]:
df.shape
print(df['title'].value_counts())
#remove singleton titles
for title in df['title']:
    if (df.title == title).sum() <= 1:
        df = df[df.title != title]
df.head()

How to Conduct a Workshop                        36
How to Be Healthy                                25
How to Give Yourself a Deep Cleansing Facial     23
How to Be Well Read                              19
How to Wash Hair in a Sink                       17
                                                 ..
How to Find the Nearest Casino1                   2
How to Find the Nearest Casino3                   2
How to Remember Your Past Lives2                  2
How to Find the Nearest Casino4                   2
How to Share a Drink Without Spreading Germs1     2
Name: title, Length: 139, dtype: int64


,overview,headline,text,sectionLabel,title
0,So you're a new or aspiring artist and your c...,\nSell yourself first.,"Before doing anything else, stop and sum up y...",Steps,How to Sell Fine Art Online
1,"If you want to be well-read, then, in the wor...",\nRead the classics before 1600.,Reading the classics is the very first thing ...,Reading the Classics,How to Be Well Read
2,So you're a new or aspiring artist and your c...,\nJoin online artist communities.,Depending on what scale you intend to sell yo...,Steps,How to Sell Fine Art Online
3,So you're a new or aspiring artist and your c...,\nMake yourself public.,Get yourself out there as best as you can by ...,Steps,How to Sell Fine Art Online
4,So you're a new or aspiring artist and your c...,\nBlog about your artwork.,"Given the hundreds of free blogging websites,...",Steps,How to Sell Fine Art Online


In [7]:
def process_instructions(dataframe) -> dict:
    # dictionary of title (article) to text (list of steps)
    wikihow = dict()

    for idx, row in df.iterrows():
        title = row['title']
        text = row['text']#.replace("\n", " ").replace("\\'", "'") + " " + row['text'].replace("\n", " ").replace("\'", "'")
        if title and text:
            if title in wikihow:
                wikihow[title].append(text)
            else:
                wikihow[title] = [text]
    
    return wikihow

In [8]:
# pairs of (instruction, rank)
def get_instruction_rank(database: dict) -> list:
    instruction_rank = list()
    
    for article, instructions in database.items():
        for idx, step in enumerate(instructions, start=1):
            instruction_rank.append((step, idx / len(instructions)))
    
    return instruction_rank

In [9]:
# pairs of (instruction, rank)
def get_instruction_rank(database: dict) -> list:
    instruction_rank = list()
    
    for article, instructions in database.items():
        for idx, step in enumerate(instructions, start=1):
            instruction_rank.append((step, idx / len(instructions)))
    
    return instruction_rank

In [65]:
# convert text instructions to a vector
def convert_vector(instruction_rank: list, num_examples: int) -> list:
    vector_rank = list()
    
    for step in instruction_rank[:num_examples]:
        instruction1, rank1,instruction2,rank2,tf = step[0][0], step[0][1],step[1][0],step[1][1],step[2]
        doc1 = nlp(instruction1)
        doc2 = nlp(instruction2)
        vector_rank.append(([doc1.vector, rank1],[doc2.vector,rank2],tf))
    
    return vector_rank

In [41]:
wikihow = process_instructions(dataframe=df)

In [42]:
sample = wikihow['How to Sell Fine Art Online']

In [43]:
def addRank(lyst):
    ranked = list()
    for index, element in enumerate(lyst):
        ranked.append((element, index))
    return ranked

In [44]:
def inOrder(lyst):
    ordered = list()
    for element in lyst:
        element = list(element)
        element.append(element[0][1] < element[1][1])
        ordered.append(tuple(element))
    return ordered

In [54]:
def makePairs(lyst):
    perms = list(permutations(addRank(lyst), 2))
    return inOrder(perms)

In [55]:
def makePairsList(wiki):
    pairslist = list()
    for k in wiki.keys():
        pairslist += makePairs(wikihow[k])
    return pairslist
pairs = makePairsList(wikihow)

In [56]:
wikihow

{'How to Sell Fine Art Online': [" Before doing anything else, stop and sum up yourself as an artist. Now, think about how to translate that to an online profile. Be it the few words, Twitter allows you or an entire page of indulgence that your own website would allow you. Bring out the most salient features of your creativity, your experience, your passion, and your reasons for painting. Make it clear to readers why you are an artist who loves art, produces high quality art, and is a true champion of art. If you're not great with words, find a friend who can help you with this really important aspect of selling online – the establishment of your credibility and reliability.;\n",
  ' Depending on what scale you intend to sell your art pieces, you may want to get an account on an online art community or store, like Deviant Art. With 15% -20\xa0% brokerage, you can also find many online art galleries like Art Brokerage, Diva Art Group, or Saatchi Art that will show your artwork online an

In [51]:
# number of articles
len(df)

993

In [11]:
instruction_rank = get_instruction_rank(database=wikihow)

In [52]:
pairs[0]

((" Before doing anything else, stop and sum up yourself as an artist. Now, think about how to translate that to an online profile. Be it the few words, Twitter allows you or an entire page of indulgence that your own website would allow you. Bring out the most salient features of your creativity, your experience, your passion, and your reasons for painting. Make it clear to readers why you are an artist who loves art, produces high quality art, and is a true champion of art. If you're not great with words, find a friend who can help you with this really important aspect of selling online – the establishment of your credibility and reliability.;\n",
  0),
 (' Depending on what scale you intend to sell your art pieces, you may want to get an account on an online art community or store, like Deviant Art. With 15% -20\xa0% brokerage, you can also find many online art galleries like Art Brokerage, Diva Art Group, or Saatchi Art that will show your artwork online and sell them to the buyer 

In [66]:
# work with 100 instructions to start off
vector_rank = convert_vector(pairs, num_examples=100)

In [67]:
print(vector_rank)

[([array([ 0.03710671,  0.31462356,  0.16997588, -1.073456  ,  0.28339627,
       -0.5951992 , -0.33690098,  0.55675197, -0.7661422 ,  0.74050397,
       -0.08414899, -0.1801639 ,  0.24979956,  0.75732106, -0.06611507,
        0.4614911 , -0.11447324, -0.71273917,  0.30717745,  0.61499494,
        0.15816507, -0.00781302,  0.5305223 , -0.51866513,  0.5530283 ,
        0.8135726 , -0.6216484 , -0.8318409 , -1.1936392 , -0.19510266,
        0.2191012 , -0.5931372 ,  0.223454  , -0.34112877, -0.7629734 ,
       -0.7872179 , -0.48325825,  0.03324556, -0.22384942, -0.3304299 ,
        1.2234806 ,  0.23740347,  0.5631622 , -0.71889687,  0.9419407 ,
       -0.56373507, -0.20121607,  0.7947912 ,  0.35887134, -0.20664403,
       -0.6904762 , -0.15493265, -0.78497666, -0.10592517, -0.11551616,
        0.91337043,  0.09081539,  0.39304715, -0.6960455 ,  0.53152555,
        0.61628157, -0.5066837 ,  0.7146497 , -0.48233727, -1.1391002 ,
        1.3350939 ,  1.0692215 , -0.7638183 ,  0.80260754,  0

       -0.38019362], dtype=float32), 10], True)]
